Загружаем датасет

In [54]:
# !pip install pydicom


import os
from pathlib import Path
import zipfile
import gdown
import pydicom
import cv2
import numpy as np
import pandas as pd
from skimage.filters import sobel, rank
from skimage.feature import hog
from skimage import exposure
from skimage.morphology import disk
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from numpy.linalg import svd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, recall_score, f1_score, fbeta_score, roc_auc_score
import pickle



# DATASET_PARENT_LOCATION = Path('.')  # UNCOMMENT WHEN IN COLAB
DATASET_PARENT_LOCATION = Path('..')  # UNCOMMENT WHEN IN REPO
DATASET_NAME = 'rsna-pneumonia-dataset'
DATASET_LOCATION = DATASET_PARENT_LOCATION / DATASET_NAME
GOOGLE_FILE_ID = '1nIW5qgn4MurehHDiulrTMHNQMpsu4SRJ'
ZIP_FILE_NAME = 'rsna-pneumonia-detection-challenge.zip'
wrong_age_entries   = [
    '3b8b8777-a1f6-4384-872a-28b95f59bf0d',
    'f632328d-5819-4b29-b54f-adf4934bbee6',
    '73aeea88-fc48-4030-8564-0a9d7fdecac4',
    'ec3697bd-184e-44ba-9688-ff8d5fbf9bbc',
    'a4e8e96d-93a6-4251-b617-91382e610fab',
    ]  # На этапе EDA у этих пациетнтов обнаружились ошибки в значении возраста
SEED = 74
np.random.seed(SEED)


def init(SIZE=128, N=200):
  if DATASET_NAME not in os.listdir(DATASET_PARENT_LOCATION):
    if ZIP_FILE_NAME not in os.listdir(DATASET_PARENT_LOCATION):
      print('Downloading the dataset')
      gdown.download(f'https://drive.google.com/uc?id={GOOGLE_FILE_ID}', ZIP_FILE_NAME, quiet=True)

    os.mkdir(DATASET_LOCATION)
    with zipfile.ZipFile(ZIP_FILE_NAME, 'r') as zip_ref:
      print('Extracting from .zip')
      zip_ref.extractall(DATASET_LOCATION)
    os.remove(ZIP_FILE_NAME)

  # create dataframes
  labels = pd.read_csv(DATASET_LOCATION / 'stage_2_train_labels.csv', usecols=['patientId', 'Target'])

  labels = labels.drop_duplicates()  # Удаляем дубликаты (у пациентов с несколькими ббоксами)
  labels = labels[~np.isin(labels['patientId'], wrong_age_entries)]  # Удаляем пациентов с неправильным значением возраста
  labels = labels.groupby('Target')[['patientId', 'Target']].apply(lambda x: x.sample(N // 2, random_state=SEED)).reset_index(drop=True).sample(N)  # Выбираем n случайных пациентов

  images = []
  print('Extracting images from DICOM')
  # for id in tqdm(labels['patientId'], desc='Extracting images from DICOM'):
  for id in labels['patientId']:
    dcm = pydicom.dcmread(DATASET_LOCATION / 'stage_2_train_images' / (id + '.dcm'))
    images.append(cv2.resize(dcm.pixel_array, (SIZE, SIZE)))

  images = np.array(images)
  labels = np.array(labels['Target'])

  return images, labels


Отбираем N изображений для обучения. Использование части датасета позволяет обучаться на сбалансированных классах

In [47]:
SIZE = 128  # Размер изображения SIZE x SIZE
N = 2000  # Количество используемых изображений
images, labels = init(N=N, SIZE=SIZE)

(f'Изображения: {images.shape}, метки классов: {labels.shape}. Доля положительного класса: {labels.sum() / labels.shape[0]}')

Extracting images from DICOM


'Изображения: (2000, 128, 128), метки классов: (2000,). Доля положительного класса: 0.5'

Делим выборку на тренировочную и тестовую части

In [48]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.33, random_state=SEED)
X_train.shape

(1340, 128, 128)

На тренировочной обучаем PCA-алгоритм для дальнейшего получения дополнительных признаков

In [49]:
n_comp = 32
pca = PCA(n_components=n_comp).fit(X_train.reshape(X_train.shape[0], -1))
pca

PCA(n_components=32)

В качестве методов предобработки были выбраны локальное выравнивание гистограммы и PCA

In [50]:
def preprocess(image_array):
    image_array = image_array / 255.
    features = []
    for img in range(image_array.shape[0]):
        cur_image = image_array[img, :, :]
        img_features = np.array([])

        footprint = disk(30)
        img_features = np.concatenate((img_features, rank.equalize(cur_image, footprint=footprint).reshape(-1)))

        features.append(img_features)

    features_array = np.array(features)
    pca_features = pca.transform(image_array.reshape(image_array.shape[0], -1))

    return np.concatenate((features_array, pca_features), axis=1)


Обрабатываем тестовую и тренировочную части

In [51]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)

X_train.shape, X_test.shape

C:\Users\user\AppData\Local\Temp\ipykernel_7236\1107039754.py:1: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  X_train = preprocess(X_train)
C:\Users\user\AppData\Local\Temp\ipykernel_7236\1107039754.py:2: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  X_test = preprocess(X_test)


((1340, 16416), (660, 16416))

Обучаем модель

In [52]:
RF_model = RandomForestClassifier(n_jobs= -1, random_state=SEED)
RF_model.fit(X_train, y_train)
pred_proba = RF_model.predict_proba(X_test)[:, 1]
pred = (pred_proba >= 0.5).astype('int')

print(pred_proba[:10])
print(pred[:10])


[0.52 0.26 0.63 0.37 0.89 0.57 0.53 0.26 0.26 0.73]
[1 0 1 0 1 1 1 0 0 1]


Получаем метрики

In [53]:
accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)
fbeta = fbeta_score(y_test, pred, beta=2)
roc_auc = roc_auc_score(y_test, pred_proba)

print(f"""
accuracy_score: {accuracy},
recall_score: {recall},
f1_score: {f1},
fbeta_score: {fbeta},
roc_auc_score: {roc_auc},
""")


accuracy_score: 0.7106060606060606,
recall_score: 0.7283950617283951,
f1_score: 0.7119155354449472,
fbeta_score: 0.7217125382262997,
roc_auc_score: 0.7836474867724867,



Сохраняем обученные модель и PCA-алгоритм

In [55]:
with open('model.pkl','wb') as f:
    pickle.dump(RF_model, f)

with open('pca.pkl','wb') as f:
    pickle.dump(pca, f)